### Import Modules

In [2]:
# Core
import pandas as pd
import numpy as np

### Import Data

File Source: 

https://www.kaggle.com/rocki37/open-university-learning-analytics-dataset

Code Source: 

https://pythonhow.com/data-analysis-with-python-pandas/

In [3]:
df_assessment = pd.read_csv('data/assessments.csv')
df_courses = pd.read_csv('data/courses.csv')
df_studentAssessment = pd.read_csv('data/studentAssessment.csv')
df_studentInfo = pd.read_csv('data/studentInfo.csv')
df_studentRegistration = pd.read_csv('data/studentRegistration.csv')
df_studentVle = pd.read_csv('data/studentVle.csv')
df_vle = pd.read_csv('data/vle.csv')

Show all dataframe columns for analysis.

Code Source: 

https://stackoverflow.com/questions/47022070/display-all-dataframe-columns-in-a-jupyter-python-notebook/47022213

In [7]:
df_studentInfo.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


How many unique students?  Student Info file has row for all courses taken by student. 

In [8]:
df_studentInfo['id_student'].nunique()

28785

In [3]:
pd.options.display.max_columns = None

### Join Datasets

Student Assessments is the master dataset and assessment score will be the predictor.

In [4]:
df_studentAssessment.head(2)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0


In [5]:
df_studentAssessment.shape

(173912, 5)

Join assessments to courses.

Code Source: 

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [6]:
df_assess_data = pd.merge(df_assessment, df_courses, how='left', 
                          on=['code_module', 'code_presentation'])

Join assess data to studentAssessment (main dataset).

In [7]:
df_assess_data = pd.merge(df_studentAssessment, df_assess_data, 
                          how='left', on=['id_assessment'])

Join assess data to studentInfo.

In [8]:
df_assess_data = pd.merge(df_assess_data, df_studentInfo, how='left', 
                          on=['code_module', 'code_presentation', 'id_student'])

Join assess data to studentRegistration.

In [9]:
df_assess_data = pd.merge(df_assess_data, df_studentRegistration, how='left', 
                          on=['code_module', 'code_presentation', 'id_student'])

In [10]:
df_assess_data.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,268,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,268,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,268,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN


Number of rows good, after joins went from 5 to 22 features.

Now to incorporate clicks on course materials.

In [11]:
df_assess_data.shape

(173912, 22)

Join studentVle to vle (virtual learning environment).

In [13]:
df_materials = pd.merge(df_studentVle, df_vle, how='left', 
                        on=['code_module', 'code_presentation', 'id_site'])

In [14]:
df_materials.head(2)

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type,week_from,week_to
0,AAA,2013J,28400,546652,-10,4,forumng,NaN,NaN
1,AAA,2013J,28400,546652,-10,1,forumng,NaN,NaN


Multiple assessments per course, just grabbing sum_click by activity type for feature engineering.

In [15]:
df_materials.drop(['date','week_from','week_to','id_site'], 
                  inplace=True, axis=1)

In [16]:
df_materials.head(2)

,code_module,code_presentation,id_student,sum_click,activity_type
0,AAA,2013J,28400,4,forumng
1,AAA,2013J,28400,1,forumng


Group and sum.

Code Source: 

https://stackoverflow.com/questions/38001491/pandas-conditionally-combine-and-sum-rows

In [17]:
df_materials_grouped = df_materials.groupby(['code_module','code_presentation','id_student',
                                             'activity_type']).sum().reset_index()

In [18]:
df_materials_grouped.head(5)

,code_module,code_presentation,id_student,activity_type,sum_click
0,AAA,2013J,11391,forumng,193
1,AAA,2013J,11391,homepage,138
2,AAA,2013J,11391,oucontent,553
3,AAA,2013J,11391,resource,13
4,AAA,2013J,11391,subpage,32


Spread out activity type as columns with click totals as the values.

Code Source: 

https://pbpython.com/pandas-pivot-table-explained.html

In [19]:
df_activity = pd.pivot_table(df_materials_grouped, values = 'sum_click', 
                             index=['code_module','code_presentation','id_student'], 
                             columns = 'activity_type').reset_index()

In [20]:
df_activity.head(5)

activity_type,code_module,code_presentation,id_student,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url
0,AAA,2013J,11391,NaN,NaN,NaN,NaN,193.0,NaN,138.0,NaN,NaN,553.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,32.0,5.0
1,AAA,2013J,28400,10.0,NaN,NaN,NaN,417.0,NaN,324.0,NaN,NaN,537.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,87.0,48.0
2,AAA,2013J,30268,NaN,NaN,NaN,NaN,126.0,NaN,59.0,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,22.0,4.0
3,AAA,2013J,31604,2.0,NaN,NaN,NaN,634.0,1.0,432.0,NaN,NaN,836.0,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,144.0,90.0
4,AAA,2013J,32885,NaN,NaN,NaN,NaN,194.0,4.0,204.0,NaN,NaN,494.0,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN,79.0,14.0


In [21]:
df_activity.shape

(29228, 23)

Join vle data to assessment data.

In [22]:
df_clicks = pd.merge(df_assess_data, df_activity, how='left', on=['code_module', 'code_presentation', 'id_student'])

In [23]:
df_clicks.shape

(173912, 42)

In [24]:
df_clicks.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,NaN,NaN,NaN,NaN,193.0,NaN,138.0,NaN,NaN,553.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,32.0,5.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,10.0,NaN,NaN,NaN,417.0,NaN,324.0,NaN,NaN,537.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,87.0,48.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,268,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,2.0,NaN,NaN,NaN,634.0,1.0,432.0,NaN,NaN,836.0,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,144.0,90.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,268,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,NaN,NaN,NaN,NaN,194.0,4.0,204.0,NaN,NaN,494.0,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN,79.0,14.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,268,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,7.0,NaN,NaN,NaN,875.0,4.0,512.0,NaN,NaN,900.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,91.0,45.0


Fill NaN's with zero for now, to do lambda function later, that will turn the zero's back to NaN's.<br>
Then will fill NaN's with average to get better model results.

In [25]:
df_clicks = df_clicks.fillna(0)

In [26]:
df_clicks.shape

(173912, 42)

Now to fix an issue with the activity type click counts.  The click numbers are repeated for all assessment id's associated to each student.  The plan was to multiply the activity type counts by the weighting for each assessment to break out the total clicks per activity type.  I did this and it produced very poor modeling results.  I realized all of the other features are repeated for each assessment but have a different assessment score.  No wonder why the models were confused!  I've decided to run my models on two different versions of the file.  The first version will contain the course material clicks and I'll consolidate the rows by student ID.  The target will be the mean score per student ID (more features, less rows).  The second version will be just the assessment data and no course material click data (less features, more rows).  You can see the issue by looking at a specific student (shown below).

In [27]:
student_11391 = df_clicks.loc[df_clicks['id_student'] == 45642
                          ]
student_11391

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url
6,1752,45642,18,0,72.0,AAA,2013J,TMA,19.0,10.0,268,F,North Western Region,A Level or Equivalent,90-100%,0-35,0,120,N,Pass,-29.0,0.0,4.0,0.0,0.0,0.0,428.0,0.0,295.0,0.0,1.0,508.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,120.0,56.0
507,1753,45642,57,0,72.0,AAA,2013J,TMA,54.0,20.0,268,F,North Western Region,A Level or Equivalent,90-100%,0-35,0,120,N,Pass,-29.0,0.0,4.0,0.0,0.0,0.0,428.0,0.0,295.0,0.0,1.0,508.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,120.0,56.0
848,1754,45642,116,0,72.0,AAA,2013J,TMA,117.0,20.0,268,F,North Western Region,A Level or Equivalent,90-100%,0-35,0,120,N,Pass,-29.0,0.0,4.0,0.0,0.0,0.0,428.0,0.0,295.0,0.0,1.0,508.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,120.0,56.0
1038,1755,45642,166,0,73.0,AAA,2013J,TMA,166.0,20.0,268,F,North Western Region,A Level or Equivalent,90-100%,0-35,0,120,N,Pass,-29.0,0.0,4.0,0.0,0.0,0.0,428.0,0.0,295.0,0.0,1.0,508.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,120.0,56.0
1463,1756,45642,227,0,73.0,AAA,2013J,TMA,215.0,30.0,268,F,North Western Region,A Level or Equivalent,90-100%,0-35,0,120,N,Pass,-29.0,0.0,4.0,0.0,0.0,0.0,428.0,0.0,295.0,0.0,1.0,508.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,120.0,56.0


Feature engineering:

Access Date - Date minus date submitted for both datasets.  Date is the final date to complete assessment and date submitted is when it was taken.

Module Length to Number of Credits Ratio - Length of module presentation in days divided by the number of credits for the module.


In [28]:
df_clicks['assess_date'] = df_clicks['date'] - df_clicks['date_submitted']
df_assess_data['assess_date'] = df_assess_data['date'] - df_assess_data['date_submitted']

df_clicks['length_no_cred_ratio'] = df_clicks['module_presentation_length'] / df_clicks['studied_credits']
df_assess_data['length_no_cred_ratio'] = df_assess_data['module_presentation_length'] / df_assess_data['studied_credits']


In [29]:
df_assess_data.head(2)

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,assess_date,length_no_cred_ratio
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,1.0,1.116667
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,-3.0,4.466667


Drop columns not needed.

In [30]:
df_clicks.drop(['id_assessment','weight','date','date_submitted'], inplace=True, axis=1)
df_assess_data.drop(['id_student','id_assessment','date','date_submitted','date_unregistration'], inplace=True, axis=1)

Only look at students that did not withdraw.

In [31]:
df_clicks = df_clicks.loc[df_clicks['final_result'] != 'Withdrawn']
df_assess_data = df_assess_data.loc[df_assess_data['final_result'] != 'Withdrawn']

print(df_clicks.shape)
print(df_assess_data.shape)

(160817, 40)
(160817, 19)


In [32]:
df_clicks['final_result'].value_counts()

Pass           106021
Fail            28466
Distinction     26330
Name: final_result, dtype: int64

In [33]:
df_assess_data['imd_band'].value_counts()

30-40%     17812
20-30%     15851
50-60%     15775
40-50%     15766
10-20      15418
70-80%     15165
60-70%     14902
80-90%     14803
90-100%    14238
0-10%      13807
Name: imd_band, dtype: int64

Only look at students have imd_band.

In [34]:
df_clicks = df_clicks.loc[df_clicks['imd_band'] != 0]
df_assess_data.dropna(subset=['imd_band'],inplace=True)

print(df_clicks.shape)
print(df_assess_data.shape)

(153537, 40)
(153537, 19)


Group all columns and take mean of dates and score.

Code Source: 

https://towardsdatascience.com/pandas-tips-and-tricks-33bcc8a40bb9

In [35]:
labels = ['id_student','is_banked','code_module','code_presentation','assessment_type','module_presentation_length','gender',
          'region','highest_education','imd_band','age_band','num_of_prev_attempts','studied_credits','disability','final_result',
         'dataplus','dualpane','externalquiz','folder','forumng','glossary','homepage','htmlactivity','oucollaborate',
         'oucontent','ouelluminate','ouwiki','page','questionnaire','quiz','repeatactivity','resource','sharedsubpage','subpage',
         'url']

df_clicks_grouped = df_clicks.groupby(labels)['assess_date','length_no_cred_ratio','date_registration','score'].mean()

In [36]:
df_clicks_grouped.shape

(37030, 4)

In [37]:
df_clicks_grouped.head(2)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,assess_date,length_no_cred_ratio,date_registration,score
id_student,is_banked,code_module,code_presentation,assessment_type,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url,,,,
6516,0,AAA,2014J,TMA,269,M,Scotland,HE Qualification,80-90%,55<=,0,60,N,Pass,21.0,0.0,0.0,0.0,451.0,0.0,497.0,0.0,0.0,1505.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,143.0,143.0,2.6,4.483333,-52.0,61.8
11391,0,AAA,2013J,TMA,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,0.0,0.0,0.0,0.0,193.0,0.0,138.0,0.0,0.0,553.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,32.0,5.0,1.8,1.116667,-159.0,82.0


Reset index of clicks file.

In [38]:
df_clicks_grouped.reset_index(inplace=True)

Dropping uneeded variables.

In [39]:
df_clicks_grouped.drop(['id_student'], inplace=True, axis=1)

In [40]:
df_clicks_grouped.shape

(37030, 38)

In [41]:
df_clicks_grouped.head(2)

,is_banked,code_module,code_presentation,assessment_type,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url,assess_date,length_no_cred_ratio,date_registration,score
0,0,AAA,2014J,TMA,269,M,Scotland,HE Qualification,80-90%,55<=,0,60,N,Pass,21.0,0.0,0.0,0.0,451.0,0.0,497.0,0.0,0.0,1505.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,143.0,143.0,2.6,4.483333,-52.0,61.8
1,0,AAA,2013J,TMA,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,0.0,0.0,0.0,0.0,193.0,0.0,138.0,0.0,0.0,553.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,32.0,5.0,1.8,1.116667,-159.0,82.0


Writing out click and assessment files.

In [42]:
df_clicks_grouped.to_csv(r'data/eda_click_data.csv',index=False)
df_assess_data.to_csv(r'data/eda_assess_data.csv',index=False)